In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama

# Define the function for the tool
def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    # Mock implementation, replace with actual API call
    weather_data = {
        "New York": "The weather in New York is 22°C with clear skies.",
        "San Francisco": "The weather in San Francisco is 15°C with fog."
    }
    return weather_data.get(location, "Weather data not available for this location.")

# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def sendMail(receiver: str, subject: str, body: str) -> str:
    """Can be used to send email/mail to any email id."""
    import smtplib
    s = smtplib.SMTP('smtp.outlook.com', 587)
    s.starttls()
    s.login("karanghai9@outlook.com", "Password")
    message = f"From: karanghai9@outlook.com\nTo: {receiver}\nSubject: {subject}\n\n{body}"
    s.sendmail("karanghai9@outlook.com", receiver, message)
    s.quit()
    return "Message sent successfully."

def opencamera():
    """Open/Launch/Start the camera whenever user asks for camera"""
    import cv2
    import time
    import json

    # Open the default camera (index 0)
    cap = cv2.VideoCapture(0)

    # Check if the camera opened successfully
    if not cap.isOpened():
        output = {
            "error": "Unable to open the camera."
        }
    else:
        # Record the start time
        start_time = time.time()
        
        # Loop to continuously capture frames from the camera
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()

            # Display the captured frame
            cv2.imshow('Camera', frame)

            # Check if 5 seconds have elapsed
            if time.time() - start_time > 5:
                output = {
                    "message": "Recording stopped after 5 seconds."
                }
                break

            # Check if the user pressed 'q' to quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                output = {
                    "message": "User stopped the recording."
                }
                break

    # Release the camera
    cap.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

    # Convert the output to JSON format
    output_json = json.dumps(output)

    # Print the output
    print(output_json)



# Initialize the Ollama LLM
llm = Ollama(model="mistral:latest", request_timeout=5000.0)

# Create the FunctionTool
weather_tool = FunctionTool.from_defaults(
    get_weather,
    # async_fn=aget_weather,  # optional!
)

multiply_tool = FunctionTool.from_defaults(multiply)

camera_tool = FunctionTool.from_defaults(opencamera)

email_tool = FunctionTool.from_defaults(sendMail)


# Make sure to pass tools as a list
tools = [weather_tool, multiply_tool, camera_tool, email_tool]

# Initialize the agent with the tools and LLM
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

# Use the agent to chat
# response = agent.chat("What is the weather in New York?")
# response = agent.chat("Can you please start the recording using camera?")
# print("Agent Response:", response)

# prompts = [
#     # "What is 50839723 times 2324?",
#     # "What is the weather of New York and San Francisco?",
#     "Multiply the temperatures of New York and San Francisco and return the result"
# ]

# for prompt in prompts:
#     response = agent.chat(prompt)
#     print("Agent Response:", response)

# response = agent.chat("Please send a mail to the receiver 'kg9811422163@gmail.com' with the body 'We have a meeting at 4 today' and the subject 'Important meeting'. ")
# print("Agent Response:", response)


In [22]:
#WITHOUT BUFFER MEMORY
from flask import Flask, request, render_template_string, send_file

app = Flask(__name__)

html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }
        .chat-container {
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 10px 10px 0px 0px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .chat-form{
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 0px 0px 10px 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .chat-message {
            background-color: #f9f9f9;
            border-radius: 10px;
            padding: 10px;
            margin: 10px 0;
        }
        .user-message {
            text-align: right;
        }
        input[type="text"] {
            width: calc(100% - 100px); /* Adjust width of input */
            padding: 8px;
            border-radius: 5px;
            border: 1px solid #ccc;
            margin-right: 10px;
        }
        .additional-content {
            display: none;
        }
        .show-button{
            background-color: #555555;
            border: none;
            color: white;
            padding: 8px 10px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 5px;
        }
        #input_submit{
            background-color: #555555;
            border: none;
            color: white;
            padding: 8px 10px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 5px;
        }
    </style>
</head>
<body>
    <div class="chat-container" id="chat-container">
    <h3>Welcome to the Artificial Intelligence QnA service.  You may ask me anything 🙃</h3>
        {% for message in chat_history %}
            <div class="chat-message {% if message['sender'] == 'user' %}user-message{% endif %}">
                {{ message['content'] }}
                {% if message['sender'] == 'bot' %}
                    <br><br>
                {% endif %}
            </div>
        {% endfor %}
    </div>
    <div class="chat-form">
    <form action="/" method="POST" id="chat-form">
        <input type="text" name="input_text" id="input_text" placeholder="Message Chatbot...">
        <input type="submit" id="input_submit" value="Send">
    </form>
    </div>
    <script>
        document.getElementById('chat-form').addEventListener('submit', function(event) {
            event.preventDefault(); // Prevent default form submission
            var inputText = document.getElementById('input_text').value;
            if (inputText.trim() !== '') {
                var inputBox = document.getElementById('input_text');
                var chatContainer = document.getElementById('chat-container');
                var userMessage = document.createElement('div');
                userMessage.className = 'chat-message user-message';
                userMessage.textContent = inputText;
                chatContainer.appendChild(userMessage);
                document.getElementById('chat-form').submit(); // Submit form
            }
        });
    });
    </script>
</body>
</html>
"""

chat_history = []

@app.route('/', methods=['GET', 'POST'])
def index():
    global chat_history

    if request.method == 'POST':
        input_text = request.form['input_text']

        response = agent.chat(input_text)
        chat_history.append({'sender': 'user', 'content': input_text})
        response_text = response
        chat_history.append({'sender': 'bot', 'content': response_text})

    return render_template_string(html_template, chat_history=chat_history)

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=91)